In [186]:
import pandas
from scipy.spatial.distance import cdist, cosine
import numpy as np

data = pandas.read_csv('movie_metadata.csv')
raw_data = data.copy()
cols_to_norm = ['num_critic_for_reviews', 'duration', 'gross', 'num_voted_users', 'num_user_for_reviews', 'title_year', 'imdb_score']
data[cols_to_norm] = data[cols_to_norm].apply(lambda x: (x - x.mean()) / (x.max() - x.min()))
data['movie_title'] = data['movie_title'].apply(lambda x: x.replace("\xc2\xa0", " ").strip())
raw_data['movie_title'] = data['movie_title'].apply(lambda x: x.replace("\xc2\xa0", " ").strip())
map_color = set(data['color'])
data


,color,director_name,num_critic_for_reviews,duration,director_facebook_likes,actor_3_facebook_likes,actor_2_name,actor_1_facebook_likes,gross,genres,...,num_user_for_reviews,language,country,content_rating,budget,title_year,actor_2_facebook_likes,imdb_score,aspect_ratio,movie_facebook_likes
0,Color,James Cameron,0.717741,0.140474,0,855,Joel David Moore,1000,0.936268,Action|Adventure|Fantasy|Sci-Fi,...,0.549759,English,USA,PG-13,237000000,0.065295,936,0.184540,1.78,33000
1,Color,Gore Verbinski,0.199268,0.122617,563,1000,Orlando Bloom,40000,0.343108,Action|Adventure|Fantasy,...,0.190794,English,USA,PG-13,300000000,0.045295,5000,0.083274,2.35,0
2,Color,Sam Mendes,0.568726,0.080950,0,161,Rory Kinnear,11000,0.199349,Action|Adventure|Thriller,...,0.142564,English,UK,PG-13,245000000,0.125295,393,0.045299,2.35,85000
3,Color,Christopher Nolan,0.828578,0.112696,22000,23000,Christian Bale,27000,0.525522,Action|Thriller,...,0.479982,English,USA,PG-13,250000000,0.095295,23000,0.260489,2.35,164000
4,NaN,Doug Walker,NaN,NaN,131,NaN,Rob Walker,131,NaN,Documentary,...,NaN,NaN,NaN,NaN,NaN,NaN,12,0.083274,NaN,0
5,Color,Andrew Stanton,0.396312,0.049204,475,530,Samantha Morton,640,0.032334,Action|Adventure|Sci-Fi,...,0.091961,English,USA,PG-13,263700000,0.095295,632,0.019983,2.35,24000
6,Color,Sam Raimi,0.310106,0.096823,0,4000,James Franco,24000,0.378777,Action|Adventure|Romance,...,0.322046,English,USA,PG-13,258000000,0.045295,11000,-0.030650,2.35,0
7,Color,Nathan Greno,0.226362,-0.014288,15,284,Donna Murphy,799,0.200313,Adventure|Animation|Comedy|Family|Fantasy|Musi...,...,0.022579,English,USA,PG,260000000,0.075295,553,0.171881,1.85,29000
8,Color,Joss Whedon,0.609367,0.067061,0,19000,Robert Downey Jr.,26000,0.539803,Action|Adventure|Sci-Fi,...,0.166877,English,USA,PG-13,250000000,0.125295,21000,0.133907,2.35,118000
9,Color,David Yates,0.289170,0.090871,282,10000,Daniel Radcliffe,25000,0.333316,Adventure|Family|Fantasy|Mystery,...,0.138413,English,UK,PG,250000000,0.065295,11000,0.133907,2.35,10000


In [189]:
d = raw_data[['movie_title', 'title_year', 'actor_1_name', 'actor_2_name', 'imdb_score']]
d2 = d.copy()
d2['title_year'] = d[['movie_title', 'title_year']].apply(lambda x: str(x[0]) + ' (' + str(x[1])[:-2] + ')', axis=1)
d2 = d2[['title_year', 'actor_1_name', 'actor_2_name', 'imdb_score']]
# d2.rename(columns={"actor_1_name": "actor1", "actor_2_name": "actor2", "imdb_score": "score"})
d2.columns =['title(year)', 'actor1', 'actor2', 'score']
d2
d2.to_json('basicinfo.js', orient='index')

# d['title_year'] = d[['movie_title', 'title_year']].apply(lambda x: str(x[0]) + ' (' + str(x[1])[:-2] + ')', axis=1)
d['search_string'] = d[['movie_title', 'title_year', 'actor_1_name', 'actor_2_name', 'imdb_score']].apply(lambda x: str(x[0]) + str(x[1])[:-2] + str(x[2]) + str(x[3]) + str(x[4]), axis=1)

d['search_string'] = d['search_string'].apply(lambda x: x.lower().replace(' ', ''))
d['search_string'].to_json('searchstr.js', orient='values')

/Users/ryecao/.pyenv/versions/2.7.11/lib/python2.7/site-packages/ipykernel/__main__.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/ryecao/.pyenv/versions/2.7.11/lib/python2.7/site-packages/ipykernel/__main__.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [165]:
from collections import Counter
c = Counter(d['title_year'])
for k, v in c.iteritems():
    if v > 1:
        print k,v

Big Fat Liar (2002) 2
Across the Universe (2007) 2
Crossroads (2002) 2
King Kong (2005) 3
Aloha (2015) 2
Lolita (1962) 2
Dekalog (n) 2
The Omen (1976) 2
Around the World in 80 Days (2004) 2
Forsaken (2015) 2
The Twilight Saga: Breaking Dawn - Part 2 (2012) 2
Skyfall (2012) 2
Snitch (2013) 2
Casino Royale (2006) 2
The French Connection (1971) 2
The Calling (2014) 2
Planet of the Apes (2001) 2
Ghostbusters (2016) 2
Oz the Great and Powerful (2013) 2
Syriana (2005) 2
Cat People (1982) 2
Unbroken (2014) 2
Hercules (2014) 2
The Return of the Living Dead (1985) 2
RoboCop (2014) 2
Glory (1989) 2
Victor Frankenstein (2015) 3
A Nightmare on Elm Street (1984) 2
Pan (2015) 3
Chasing Liberty (2004) 2
Bad Moms (2016) 2
The Gambler (2014) 2
Spider-Man 3 (2007) 2
Heist (2015) 2
Conan the Barbarian (1982) 2
Dredd (2012) 2
The Full Monty (1997) 2
Alice in Wonderland (2010) 2
Twilight (2008) 2
The Claim (2000) 2
Creepshow (1982) 2
My Soul to Take (2010) 2
Unknown (2011) 2
Sabotage (2014) 2
A Woman, a Gu

In [90]:
data = data.as_matrix()

In [116]:
def dist(m1, m2):
    res = 0
    color_weight = 0.1
    director_weight = 3
    actor_weight = 1 
    genre_weight = 0.2 
    plot_weight = 0.2 
    language_weight = 0.1
    country_weight = 0.1
    metrics_weight = 2
    
    # color
    if m1[0] == m2[0]:
        res += color_weight
    
    # director name
    if m1[1] == m2[1]:
        res += director_weight

    actors_1, actors_2 = set([m1[6], m1[10], m1[14]]), set([m2[6], m2[10], m2[14]])
    res += actor_weight * len(actors_1 & actors_2)
    
    genres_1, genres_2 = set(str(m1[9]).split('|')), set(str(m2[9]).split('|'))
    res += genre_weight * len(genres_1 & genres_2)
    
    plot_1, plot_2 = set(str(m1[16]).split('|')), set(str(m2[16]).split('|'))
    res += plot_weight * len(plot_1 & plot_2)
    
    # language
    if m1[19] == m2[19]:
        res += language_weight

    # country
    if m1[20] == m2[20]:
        res += country_weight
    
    # num_critic_for_reviews, duration, gross, num_voted_users, num_user_for_reviews, title_year, imdb_score
    v1 = np.nan_to_num(np.array([m1[2], m1[3], m1[8], m1[12], m1[18], m1[23], m1[25]]))
    v2 = np.nan_to_num(np.array([m2[2], m2[3], m2[8], m2[12], m2[18], m2[23], m2[25]]))
    
    res += metrics_weight * (1 + cosine(v1, v2))
    
    return res

In [105]:
data[0][16]

'avatar|future|marine|native|paraplegic'

In [117]:
from heapq import *

h =[]
for i in xrange(1, len(data)):
    heappush(h, (-dist(data[0], data[i]), data[i][11]))

print heappop(h), heappop(h), heappop(h), heappop(h)

(-7.6047095067181179, 'The Abyss') (-7.3709398428967674, 'Deep Rising') (-7.1951998055702964, 'Undisputed') (-7.1074424458792693, 'Tales from the Crypt: Demon Knight')


In [123]:
l[1][1]

0